# Segmenting and Clustering Neighborhoods in Toronto
## Attaching latitude and longitude to postal codes
#### Postalcodes from previous assignment were saved into csv files and read for further use

In [20]:
# read the postal codes we prepared in the previous assignment
import pandas as pd
df_TN_1 = pd.read_csv('PostalCodesToronto.csv')

In [21]:
df_TN_1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Lets find the Latitude and Longitude for 1 postal code

In [9]:
# We have to install the geocoder
!pip install geocoder

     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011
     |████████████████████████████████| 92kB 13.2MB/s eta 0:00:01
     |████████████████████████████████| 829kB 20.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [19]:
import geocoder # import geocoder
# initialize your variable to None
lat_lng_coords = None
# lets count the times trying
times = 1
# loop until you get the coordinates
while(lat_lng_coords is None):
    times = times + 1
    if times == 2500: break
    g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
    lat_lng_coords = g.latlng

if lat_lng_coords is None:
    print('Not found within 2500 times')
else:
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print('latitude = ', latitude, ' found in ', times , ' times trying' )

Not found within 2500 times


#### investigated further, and found more students with problems, so I assumed the geocoder doesn't work (anymore)
#### Quote from  Thomas Buccelli:
"No returns on any coords for using geocoder
 Thomas Buccelli Technical Lab Issues · 4 days ago
I am not sure if geocoder is "down", but even using the sample code provided, my queries run indefinitely. 
I am running on Watson Studio and break these after 100 or so attempts as indicated by the attempt print message. Code is as below.

# Lets try another geocode module
### pgeocode


Postal code geocoding and distance calculations

pgeocode is a Python library for high performance off-line querying of GPS coordinates, region name and municipality name from postal codes. Distances between postal codes as well as general distance queries are also supported. The used GeoNames database includes postal codes for 83 countries.

In [22]:
!pip install pgeocode

#### Lets define 2 DataFrames for Latitude and Longitude
##### with the same length as the postal codes DataFrame

In [24]:
lat = pd.DataFrame({'Latitude': range(103)})
long = pd.DataFrame({'Longitude': range(103)})
df_TN_1.shape

(103, 3)

In [25]:
import pgeocode
nomi = pgeocode.Nominatim('ca')

i=0
while i < 103:
    PCQ = nomi.query_postal_code(df_TN_1.iloc[i,0]) 
    lat.iloc[i] = PCQ.latitude
    long.iloc[i] = PCQ.longitude
    i=i+1

df_TN_ll = pd.concat([df_TN_1,lat,long],axis=1)
df_TN_ll.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


#### While this wordked, the results are quite a bit different from the provided csv file
#### So I opted to use that as well.
##### First get the data and store it in a dataframe with a good index

In [26]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo_i = df_geo.set_index('Postal Code')
df_geo_i.head()

--2020-04-30 12:57:32--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 158.85.108.83, 158.85.108.86, 169.48.113.194
Connecting to cocl.us (cocl.us)|158.85.108.83|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-30 12:57:34--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-30 12:57:35--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr8

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


##### Now replace the Latitude and Longitude from this dataframe (df_geo_i) to the original dataframe (df_TN_ll)


In [27]:
i=0
while i < 103:
    PC = df_TN_ll.iloc[i,0]
    df_TN_ll.iloc[i,3] = '{:2.6f}'.format(df_geo_i.loc[PC, 'Latitude'])
    df_TN_ll.iloc[i,4] = '{:2.6f}'.format(df_geo_i.loc[PC, 'Longitude'])
    i=i+1

df_TN_ll.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
